In [4]:
!pip install torch

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 44.8 MB/s  0:00:01m0:00:0100:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [torch]32m8/9 [torch]]x]


In [6]:
import os
import h5py
import torch
from torch.utils.data import Dataset, ConcatDataset

# ---- Your existing dataset class ----
class BrainToTextDataset(Dataset):
    def __init__(self, filepath, require_labels=True):
        self.filepath = filepath
        self.trials = []
        self.require_labels = require_labels

        with h5py.File(filepath, "r") as f:
            for tkey in sorted(f.keys()):
                feats = f[tkey]["input_features"][()]

                if "seq_class_ids" in f[tkey]:
                    labels = f[tkey]["seq_class_ids"][()]
                else:
                    if require_labels:
                        # Skip trials with no labels if we expect them
                        continue
                    else:
                        labels = np.array([])  # placeholder

                self.trials.append((feats, labels))

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        x, y = self.trials[idx]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long) if len(y) > 0 else torch.tensor([])
        return x, y


# ---- Function to gather datasets across all days ----
def load_all_days(base_dir):
    train_sets, val_sets, test_sets = [], [], []

    for day in sorted(os.listdir(base_dir)):
        day_path = os.path.join(base_dir, day)
        if not os.path.isdir(day_path):
            continue

        train_file = os.path.join(day_path, "data_train.hdf5")
        val_file   = os.path.join(day_path, "data_val.hdf5")
        test_file  = os.path.join(day_path, "data_test.hdf5")

        if os.path.exists(train_file):
            train_sets.append(BrainToTextDataset(train_file, require_labels=True))
        if os.path.exists(val_file):
            val_sets.append(BrainToTextDataset(val_file, require_labels=True))
        if os.path.exists(test_file):
            test_sets.append(BrainToTextDataset(test_file, require_labels=False))  # test has no labels

    train_ds = ConcatDataset(train_sets) if train_sets else None
    val_ds   = ConcatDataset(val_sets) if val_sets else None
    test_ds  = ConcatDataset(test_sets) if test_sets else None

    return train_ds, val_ds, test_ds



# ---- Usage ----
base_dir = "data/brain-to-text-25/t15_copyTask_neuralData/hdf5_data_final"
train_ds, val_ds, test_ds = load_all_days(base_dir)

print("Total trials across all days:")
print("Train:", len(train_ds) if train_ds else 0)
print("Val:", len(val_ds) if val_ds else 0)
print("Test:", len(test_ds) if test_ds else 0)


Total trials across all days:
Train: 8072
Val: 1426
Test: 1450
